# Simulado para AF de MEGADADOS

A prova tem duração de **2h00**. Veja mais informações no Blackboard!

Não se esqueça de **anexar o notebook** no **Blackboard** ao final da prova. Deixe as células **executadas**!

**NOME**: SEU NOME AQUI!

## Importante

No VSCode ou ao utilizar o Docker (Spark) **abra apenas a pasta da prova**. É proibido abrir a pasta das aulas.

Sugestão: Crie uma pasta e copie o `.env` para lá (assim não precisará da pasta aulas). Crie um **venv** nesta pasta e instale as dependências da prova.

```console
pip install -r requirements.txt
```

Já deixe seu ambiente (pasta com `.env` + **veenv**) preparado para a prova, de tal forma que no dia dia prova baste copiar o conteúdo da prova para a pasta já preparada.

## Spark

Comandos para inicialização do **Spark**. Para macOS e linux, utilize:

```bash
docker run \
    -it \
    --rm \
    -p 8888:8888 \
    -p 4040:4040 \
    -v "`pwd`":/home/jovyan/work \
    jupyter/pyspark-notebook


```

Se estiver no Windows estes comandos, utilize:

- No Powershell: `docker run -it --rm -p 8888:8888 -p 4040:4040 -v ${PWD}:/home/jovyan/work jupyter/pyspark-notebook`

- No Prompt de comando: `docker run -it --rm -p 8888:8888 -p 4040:4040 -v %cd%:/home/jovyan/work jupyter/pyspark-notebook`

Agora abra esse notebook lá no container!

## Insper autograding!

Para receber feedback dos exercício, iremos utilizar o `insper autograding`.

In [1]:
# !pip install -U git+https://github.com/macielcalebe/insperautograding.git

## Como resolver os exercícios?

Crie a base da prova em sua máquina (Exercício 1). Utilize o notebook, MySQL Workbench ou o conector para testar as queries e soluções. Quando estiver bastante certo de que a resposta está correta, faça a submissão para o servidor.

## Base de dados

Em alguns exercícios, iremos utilizar a base de dados `af_md_23_2`.

Execute o script `mensagens.sql` no Workbench para criar e popular a base.

## Import das bibliotecas

Vamos realizar o import das bibliotecas.

In [1]:
import mysql.connector
import os
import insperautograder.jupyter as ia
from functools import partial
from dotenv import load_dotenv

E vamos criar nosso HELPER de conexão com o banco! Perceba que, uma vez configurado o `.env` não precisaremos mais informar usuários, senhas e URLs!

In [ ]:
load_dotenv(override=True)

def get_connection_helper(database):

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            results = cursor.execute(query, args, multi=True)
            for i, result in enumerate(results):
                if result.with_rows:
                    print(f"Resultado query {i}:")
                    for line in result.fetchall():
                        print(line)
                else:
                    print(f"Query {i} executada!")

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database=database,
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper("af_md_23_2")

In [ ]:
# !pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MinhaAplicacao").getOrCreate()
sc = spark.sparkContext
sc

### Notas

As primeiras cinco questões, que possuem correção automática, valem 60% da nota da prova.

Na API de correção automática a nota de cada questão será ponderada pelo seu peso. A nota será apresentada no intervalo 0 a 10, multiplique por 0.6 para saber a nota final considerando toda a prova.

Para conferir a nota da correção automática da prova, utilize:

In [2]:
ia.grades(task="af_md_23_2")

|    | Atividade   | Exercício   |   Peso |   Nota |   Nota Sem Atraso |   Nota Com Atraso |
|---:|:------------|:------------|-------:|-------:|------------------:|------------------:|
|  0 | af_md_23_2  | ex01spark   |      2 |      0 |                 0 |                 0 |
|  1 | af_md_23_2  | ex02spark   |      3 |      0 |                 0 |                 0 |
|  2 | af_md_23_2  | ex03trigger |      3 |      0 |                 0 |                 0 |
|  3 | af_md_23_2  | ex04perm    |      1 |      0 |                 0 |                 0 |
|  4 | af_md_23_2  | ex05python  |      2 |      0 |                 0 |                 0 |

In [3]:
ia.grades(by="TASK", task="af_md_23_2")

|    | Tarefa     |   Nota | Conta como ATV?   |
|---:|:-----------|-------:|:------------------|
|  0 | af_md_23_2 |      0 | Sim               |

In [4]:
ia.grades(by="TASK")

|    | Tarefa               |   Nota | Conta como ATV?   |
|---:|:---------------------|-------:|:------------------|
|  0 | newborn              |  10    | Não               |
|  1 | select01             |  10    | Sim               |
|  2 | ddl                  |  10    | Sim               |
|  3 | dml                  |  10    | Sim               |
|  4 | agg_join             |  10    | Sim               |
|  5 | group_having         |  10    | Sim               |
|  6 | views                |  10    | Sim               |
|  7 | sql_review1          |  10    | Sim               |
|  8 | permissions          |  10    | Sim               |
|  9 | ai_md_23_1           |  10    | Sim               |
| 10 | ai_md_23_2           |  10    | Sim               |
| 11 | ai_md_25_2           |   6.88 | Não               |
| 12 | desafio_normalizacao |  10    | Não               |
| 13 | triggers             |  10    | Sim               |
| 14 | functional           |  10    | Sim               |
| 15 | exercicios_spark     |  10    | Sim               |
| 16 | spark                |  10    | Sim               |
| 17 | af_md_23_2           |   0    | Sim               |
| 18 | revisao_af_md_1      |  10    | Sim               |

In [5]:
ia.average(excluded_count=2)

|    |   Média de ATV |
|---:|---------------:|
|  0 |             10 |

**Exercício 1**:

O arquivo `data/timestamps.txt` contém uma lista de timestamps indicando quando foram feitos acessos a um sistema, um por linha. Cada timestamp é uma string no formato `YYYY-MM-DD HH:MM:SS`. Escreva uma função em Spark que recebe um rdd e um ano (nesta ordem) e retorna quantos acessos houveram em um determinado ano.

In [6]:
def total_acessos(rdd, ano):
    # Converte o ano para string, pois cada linha começa com "YYYY"
    ano_str = str(ano)
    
    # Filtra apenas as linhas que começam com o ano desejado
    acessos_no_ano = rdd.filter(lambda linha: linha.startswith(ano_str))
    
    # Conta quantos registros sobraram após o filtro
    return acessos_no_ano.count()

rdd = sc.textFile("data/timestamps.txt")

print(total_acessos(rdd, "2022"))
print(total_acessos(rdd, "2023"))

NameError: name 'sc' is not defined

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [7]:
ia.sender(answer="total_acessos", task="af_md_23_2", question="ex01spark", answer_type="pycode")

interactive(children=(Button(description='Enviar ex01spark', style=ButtonStyle()), Output()), _dom_classes=('w…

**Exercício 2**:

Ainda no arquivo `data/timestamps.txt`, escreva uma função em Spark que recebe um rdd e retorna em ordem decrescente as 20 datas em que houveram mais acessos junto da quantidade de acessos em cada data.

In [12]:
def top20(rdd):
    # Extrai apenas a parte da data (antes do espaço)
    datas = rdd.map(lambda linha: linha.split(" ")[0])
    
    # Mapeia cada data para o par (data, 1)
    pares = datas.map(lambda d: (d, 1))
    
    # Reduz somando as ocorrências por data
    contagens = pares.reduceByKey(lambda a, b: a + b)
    
    # Ordena por número de acessos (decrescente)
    ordenado = contagens.sortBy(lambda x: x[1], ascending=False)
    
    # Retorna os 20 primeiros
    return ordenado.take(20)

top20(rdd)

NameError: name 'rdd' is not defined

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [13]:
ia.sender(answer="top20", task="af_md_23_2", question="ex02spark", answer_type="pycode")

interactive(children=(Button(description='Enviar ex02spark', style=ButtonStyle()), Output()), _dom_classes=('w…

**Exercício 3**:

Vimos na primeira parte do curso como podemos utilizar índices para acelerar a performance de *queries*. RDBMS como o MySQL suportam, inclusive, a construção de índices por múltiplas colunas.

Entretanto, em certas situações, como quando são realizadas primariamente buscar por igualdade em múltiplas colunas, uma alternativa é utilizar **hashing** de índices compostos. Nesta técnica, ao invés de construir um índice composto por múltiplas colunas e realizar as consultas neste formato:

```mysql
SELECT t.res1,
       t.res2
FROM tabela t
WHERE t.col1 = 'algum valor'
  AND t.col2 = 'outro valor'
  AND t.col3 = 'criterio de busca'
```

É construida manualmente uma coluna que contém o **hash da concatenação das colunas utilizadas na query**. Então, é criado um índice apenas na coluna que contém o hash. Assim, a *query* de consulta passa a ser, por exemplo:


```mysql
SELECT t.res1,
       t.res2
FROM tabela t
WHERE t.coluna_hash = MD5(CONCAT('algum valor', 'outro valor', 'criterio de busca'))
```

Neste exercício, você deve construir uma *trigger* para atualizar a coluna chamada `coluna_hash` da tabela de `mensagens` quando ocorrer um **update** na tabela de mensagens.

A tabela estará assim:
| id | col1    | col2     | col3    | mensagem                                 |
|----|---------|----------|---------|------------------------------------------|
| 1  | ANA     | SILVA    | ABACATE | Oi, bom dia                              |
| 2  | MARIA   | PEREIRA  | PERA    | Me liga quando puder                     |
| 3  | JOSE    | FERREIRA | ABACATE | Cade vc?                                 |
| 4  | ANTONIO | SILVA    | MACA    | Qual o status do pedido?                 |
| 5  | CAMILA  | WOK      | PERA    | Conseguiu analisar?                      |
| 6  | DANILO  | SCHINLE  | MACA    | Tudo certo pro jantar, vc vem?           |
| 7  | PEDRO   | PACKEL   | BANANA  | Complicado, vai ter que conferir com ela |
| 8  | MANOELA | NANTES   | PERA    | O lugar era lindo                        |
| 9  | CELSO   | ALIVERRI | ABACATE | A visao eh adequada                      |
| 10 | ELLEN   | WETLKS   | UVA     | Vamos juntos                             |

Por exemplo, após criar a coluna `coluna_hash`, ao fazer um *update* no id `1`, ficará assim:

| id | col1    | col2     | col3    | mensagem                                 | coluna_hash                              |
|----|---------|----------|---------|------------------------------------------|------------------------------------------|
| 1  | ANA     | SILVA    | UVA     | Oi, bom dia                              | 2d0b14a194488b9080e080c8a7d82ef88fddbe1e |
| 2  | MARIA   | PEREIRA  | PERA    | Me liga quando puder                     |                                          |
| 3  | JOSE    | FERREIRA | ABACATE | Cade vc?                                 |                                          |
| 4  | ANTONIO | SILVA    | MACA    | Qual o status do pedido?                 |                                          |
| 5  | CAMILA  | WOK      | PERA    | Conseguiu analisar?                      |                                          |
| 6  | DANILO  | SCHINLE  | MACA    | Tudo certo pro jantar, vc vem?           |                                          |
| 7  | PEDRO   | PACKEL   | BANANA  | Complicado, vai ter que conferir com ela |                                          |
| 8  | MANOELA | NANTES   | PERA    | O lugar era lindo                        |                                          |
| 9  | CELSO   | ALIVERRI | ABACATE | A visao eh adequada                      |                                          |
| 10 | ELLEN   | WETLKS   | UVA     | Vamos juntos                             |                                          |

**Obs**:
- Execute o script `mensagens.sql` para construir a base de dados;
- Faça o DDL, em seu servidor local, para criar a coluna `coluna_hash` com array `CHAR` de tamanho 40. No servidor de testes esta coluna já existe e você não deve enviar esta parte;
- Imagine que o índice já existe na coluna de hash (não é relevante para o exercício que ele realmente exista);
- A coluna `coluna_hash` deve ser calculada com o uso da função `SHA1` a partir da concatenação dos colunas `col1`, `col2` e `col3` (nesta ordem e sem espaçadores).


In [17]:
sql_ex03 = """
DROP TRIGGER IF EXISTS trg_update_coluna_hash;

CREATE TRIGGER trg_update_coluna_hash
BEFORE UPDATE ON mensagens
FOR EACH ROW
BEGIN
  -- Recalcula o hash apenas se alguma das colunas usadas mudou
  IF (NEW.col1 <> OLD.col1) OR (NEW.col2 <> OLD.col2) OR (NEW.col3 <> OLD.col3) THEN
    SET NEW.coluna_hash = SHA1(CONCAT(NEW.col1, NEW.col2, NEW.col3));
  END IF;
END;
"""

# db(sql_ex03)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [18]:
ia.sender(answer="sql_ex03", task="af_md_23_2", question="ex03trigger", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex03trigger', style=ButtonStyle()), Output()), _dom_classes=(…

**Exercício 04**:

Considere que foi criado o usuário `u_consult` com login a partir de qualquer host.

O usuário possui permissão de **LEITURA** e **INSERÇÃO** na tabela de `mensagens` utilizada no exercício anterior. Entretanto, ele não deveria poder **INSERIR** na tabela `mensagens`.

Faça as atualizações, revogando as permissões não mais necessárias.

In [19]:
sql_ex04 = """
REVOKE INSERT ON af_md_23_2.mensagens FROM 'u_consult'@'%';
"""

# db(sql_ex04)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [20]:
ia.sender(answer="sql_ex04", task="af_md_23_2", question="ex04perm", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex04perm', style=ButtonStyle()), Output()), _dom_classes=('wi…

**Exercício 5**:

Um pesquisador irá analisar parte dos dados de um projeto científico para identificação de exoplanetas.

Os dados do projeto são armazenados em um servidor com quatro HDs de 10TB configurados em RAID 5.

Durante o projeto, surgiram questões como:

*"Considerando que são recebidos 14 GB por semana, quantos dias serão necessários para que se atinja 75% da capacidade disponível para armazenamento no servidor?"*

Para responder questões como esta rapidamente, você deve construir uma função `calc_dias_armazenamento` em python que recebe, nesta ordem:

- `tb_por_hd`: quantidade de terabytes de cada um dos quatro HDs;
- `gb_semana`: quantidade de novos dados recebidos a cada semana, em GB;
- `perc`: capacidade percentual limite que se quer analisar. O percentual será passado no intervalo `[0.0 ... 1.0]`.

Como resposta, você deve devolver a quantidade inteira de dias até que que a capacidade percentual seja atingida.

**Obs**:
- Considere o sistema internacional de unidades, ou seja, 1TB = 1000GB e 1GB=1000MB.

In [21]:
def calc_dias_armazenamento(tb_por_hd, gb_semana, perc):
    # Capacidade total útil (RAID 5: n-1 discos disponíveis)
    total_gb = (4 - 1) * tb_por_hd * 1000  # converte TB -> GB
    
    # Capacidade limite em GB
    limite_gb = total_gb * perc
    
    # Número de semanas até atingir o limite
    semanas = limite_gb / gb_semana
    
    # Converter semanas para dias
    dias = semanas * 7
    
    # Retorna valor inteiro
    return int(dias)

In [22]:
# Alguns testes locais
assert calc_dias_armazenamento(10, 14000, 0.5) == 7
assert calc_dias_armazenamento(20, 12800, 0.9) == 29

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [23]:
ia.sender(answer="calc_dias_armazenamento", task="af_md_23_2", question="ex05python", answer_type="pycode")

interactive(children=(Button(description='Enviar ex05python', style=ButtonStyle()), Output()), _dom_classes=('…

**Exercício 6**: (**Nota: 1,5**)

Na atividade da próxima *sprint*, foi atribuido ao seu time de analistas de negócio a entrega de um *Dashboard* para acompanhamento dos números de vendas da empresa (vendas por região, vendas por categoria de produto, vendas por canal, etc.)

Dado que isolamento é uma das propriedades fundamentais de bancos de dados relacionais e considerando que parte da especificação envolve:

- O acompanhamento será feito em tempo real;
- É ok ocorrerem imprecisões momentâneas.

Justifique **qual nível de isolamento de transações** você recomendaria que fosse utilizado no projeto.

Esta questão será corrigida considerando a seguinte rubrica:

| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">

O nível de isolamento mais adequado é Read Committed (padrão de diversos SGBDs) ou, se houver suporte e necessidade de máxima velocidade, Read Uncommitted. Esses níveis permitem alta concorrência com baixa contenção, algo essencial para dashboards.

O isolamento Read Uncommitted é o mais baixo nível de isolamento, permitindo dirty read: ler dados ainda não confirmados (pode ser revertido depois), non-repeatable read e phantom read. Quando usar: Sistemas de monitoramento em tempo real, onde “melhor velocidade do que precisão absoluta”; Consultas analíticas leves com muita concorrência.

O isolamento Read Committed é o isolamento mais utilizado em sistemas OLTP, permitindo non-repeatable read e phantom read, mas não permite dirty reads Quando usar: SSistemas que precisam de integridade básica, mas com boa performance; A grande maioria dos dashboards, APIs de consulta, sistemas OLTP.

O isolamento Repeatable Read evita leituras não repetíveis, permitindo phantom read, mas não permite dirty reads e non-repeatable. Quando usar: Processos analíticos que precisam consistência durante uma única consulta longa.

O isolamento Serializable é de mais alto nível — simula execução sequencial, não permite dirty reads, non-repeatable e phantom read. 

</div>

**Exercício 7**:

Considere um cenário onde projetos são propostos por empresas em busca de financiamento. Cada projeto é proposto por apenas uma empresa, mas a empresa pode propor muitos projetos.

Os projetos podem ou não receber apoio financeiro. Esta decisão é tomada por meio de uma votação, onde usuários membros de um grupo qualquer decidirão se apoiam ou não cada projeto. Cada usuário vota apenas uma vez em cada projeto (`S: sim`, `N: não`, `A: anula`).

Considerando esta tabela de `votos`:

| id_usuario(PK) | nome           | id_projeto(PK) | projeto     | descricao                              | id_empresa | empresa | voto |
|----------------|----------------|----------------|-------------|----------------------------------------|------------|---------|------|
| 20             | Antonio Soares | 1              | Sockerator  | Emparelhador de meias                  | 50         | UniX    | S    |
| 21             | Marina Pereira | 1              | Sockerator  | Emparelhador de meias                  | 50         | UniX    | S    |
| 22             | Camila Mert    | 1              | Sockerator  | Emparelhador de meias                  | 50         | UniX    | N    |
| 20             | Antonio Soares | 2              | Snacktapult | Lança petiscos diretamente para a boca | 51         | Abx     | S    |
| 22             | Camila Mert    | 2              | Snacktapult | Lança petiscos diretamente para a boca | 51         | Abx     | N    |
| 20             | Antonio Soares | 3              | LazyHat     | Um boné com apoio para petiscos        | 50         | UniX    | A    |
| 21             | Marina Pereira | 3              | Lazy Hat    | Um boné com apoio para petiscos        | 50         | UniX    | A    |

Chamamos de normalização o processo de rearranjar o banco de dados para que obedeça às várias formas normais. Assim, reduzimos a redundância, aumentamos o desempenho e a integridade de dados.

**a)** (**Nota: 1,0**) Ao analisar a tabela, um aluno fez a seguinte afirmação:

*"A tabela de `votos` está na `2NF` mas não está na `1NF`"*

A afirmação do aluno está correta? Justifique detalhadamente.

Esta questão será corrigida considerando a seguinte rubrica:

| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">

A afirmação do aluno está incorreta. Pois, para que um conjunto de dados esteja dentro da normalização 2NF, é necessário que ela obedeça a normalização 1NF. Essa lógica também se aplica a 3NF, que, para ser considerada normalização 3NF, é necessário obedecer as regras da 2NF e, cpnsequentemente, da 1NF. Assim, um conjunto de dados só dados só pde estar na normalização 3NF se, e somente se, estiver na 2NF e na 1NF, e só pode estar na @NF se, e somente se, estiver na 1NF.

Um exemplo de uso comum dessas normalizações pode ser observado no caso da relação entre usuário e seus dados, como no caso de telefone e endereços dos usuários.

</div>

**b**) (**Nota: 1,5**) Baseado na tabela denormalizada fornecida, normalize os dados até a 3ª Forma Normal (`3NF`), garantindo:

1. Eliminação de redundâncias.
2. Integridade referencial entre as entidades.
3. Relacionamento correto entre jogos, times, jogadores e técnicos.

Crie no **MySQL Workbench** ou **Draw.io** o **Diagrama do modelo relacional (EER Diagram)** que representa a base de dados normalizada.

**Instruções:**
- Salve o DER na pasta `resposta_der`. Utilize formato **png** ou **jpg**.
- Insira o caminho do arquivo no notebook usando uma tag para exibir a imagem.

**Critérios de Avaliação (Rubrica):**

| Conceito | Nota Porcentual | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Insuficiente, as modificações propostas não consideram as informações a serem armazenadas ou consideram um cenário diferente do proposto |
| D        | 0.3 |Não está na 1NF, mas a solução está em desenvolvimento e permite armazenar boa parte das informações |
| C        | 0.6 |Está na 1NF, sem erros, redundâncias ou ineficiências | |
| B        | 0.8 |Está na 2NF, sem erros, redundâncias ou ineficiências | |
| A        | 1.0 |Está na 3NF, sem erros, redundâncias ou ineficiências |

<div class="alert alert-success">

Seu diagrama do workbench AQUI!

<img src="resposta_der/exemplo.png">

</div>

### Conferindo as notas!

In [24]:
ia.grades(task="af_md_23_2")

|    | Atividade   | Exercício   |   Peso |   Nota |   Nota Sem Atraso |   Nota Com Atraso |
|---:|:------------|:------------|-------:|-------:|------------------:|------------------:|
|  0 | af_md_23_2  | ex01spark   |      2 |     10 |                10 |                 0 |
|  1 | af_md_23_2  | ex02spark   |      3 |     10 |                10 |                 0 |
|  2 | af_md_23_2  | ex03trigger |      3 |     10 |                10 |                 0 |
|  3 | af_md_23_2  | ex04perm    |      1 |     10 |                10 |                 0 |
|  4 | af_md_23_2  | ex05python  |      2 |     10 |                10 |                 0 |

In [25]:
ia.grades(by="TASK", task="af_md_23_2")

|    | Tarefa     |   Nota | Conta como ATV?   |
|---:|:-----------|-------:|:------------------|
|  0 | af_md_23_2 |     10 | Sim               |

In [26]:
ia.grades(by="TASK")

|    | Tarefa               |   Nota | Conta como ATV?   |
|---:|:---------------------|-------:|:------------------|
|  0 | newborn              |  10    | Não               |
|  1 | select01             |  10    | Sim               |
|  2 | ddl                  |  10    | Sim               |
|  3 | dml                  |  10    | Sim               |
|  4 | agg_join             |  10    | Sim               |
|  5 | group_having         |  10    | Sim               |
|  6 | views                |  10    | Sim               |
|  7 | sql_review1          |  10    | Sim               |
|  8 | permissions          |  10    | Sim               |
|  9 | ai_md_23_1           |  10    | Sim               |
| 10 | ai_md_23_2           |  10    | Sim               |
| 11 | ai_md_25_2           |   6.88 | Não               |
| 12 | desafio_normalizacao |  10    | Não               |
| 13 | triggers             |  10    | Sim               |
| 14 | functional           |  10    | Sim               |
| 15 | exercicios_spark     |  10    | Sim               |
| 16 | spark                |  10    | Sim               |
| 17 | af_md_23_2           |  10    | Sim               |
| 18 | revisao_af_md_1      |  10    | Sim               |

In [27]:
ia.average(excluded_count=2)

|    |   Média de ATV |
|---:|---------------:|
|  0 |             10 |

## Entrega!

É hora de entregar. Envie um **ZIP** (não **RAR**) e finalize o teste no proctorio!